In [1]:
import numpy as np
import pandas as pd
import importlib
import random 
from time import time
import matplotlib.pyplot as plt
from scipy.spatial import distance
plt.rc('figure', figsize=(8, 5))
import helpers as h

In [20]:
def msls(points, distances, it=100):
    start = time()
    cycles = [h.steepest_search(points, distances) for _ in range(it)]
    scores = [h.score(distances, c) for c in cycles]
    return cycles[np.argmin(scores)], time() - start

def change_vertices(cycle1, cycle2, distances):
    i = random.randint(0, len(cycle1)-1)
    j = random.randint(0, len(cycle2)-1)
    return h.change_vertices(cycle1, cycle2, i, j, distances)

def change_edges_inside(cycle, distances):
    i = random.randint(0, len(cycle)-1)
    j = random.randint(0, len(cycle)-1)
    return h.change_edges_inside_1(cycle, i, j, distances)

def small_perturbation(cycle1, cycle2, n, distances):
    per = [random.choice((change_vertices, change_edges_inside)) for _ in range(n)]
    for p in per:
        if p == change_edges_inside:
            c = random.choice((cycle1, cycle2))
            c = p(c, distances)
        else:
            cycle1, cycle2, _ = p(cycle1, cycle2, distances)    
    return cycle1, cycle2

def ils_small_perturbation(points, distances, max_time, n=10):
    start = time()
    cycle1, cycle2 = h.random_solution(points, distances)
    best_score = h.score(distances, (cycle1, cycle2))
    while time() - start < max_time:
        cycle1_copy, cycle2_copy = cycle1.copy(), cycle2.copy()
        cycle1_copy, cycle2_copy = small_perturbation(cycle1_copy, cycle2_copy, n, distances)
        cycles = h.steepest_search(points, distances, cycle1_copy, cycle2_copy)
        score = h.score(distances, cycles)
        if score < best_score:
            cycle1, cycle2 = cycles
            best_score = score
    return cycle1, cycle2

def big_perturbation(cycle1, cycle2, points, distances, fraction=0.2):
    fraction = fraction / 2.0
    remaining = []
    for cycle in (cycle1, cycle2):
        n = len(cycle)
        delete = n * fraction
        start = random.randint(0, n-1)
        remaining.extend(cycle[start : start + delete])
        cycle[start : start + delete] = []
        if start + delete > n:
            remaining.extend(cycle[0 : start + delete - n])
            cycle[0 : start + delete - n] = []
    return h.regret_method(points, distances, cycle1, cycle2, remaining)
        


def ils_big_perturbation(points, distances, max_time):
    start = time()
    cycle1, cycle2 = h.random_solution(points, distances)
    best_score = h.score(distances, (cycle1, cycle2))
    while start - time() < max_time:
        cycle1_copy, cycle2_copy = cycle1.copy(), cycle2.copy()
        cycle1_copy, cycle2_copy = big_perturbation(cycle1_copy, cycle2_copy, points, distances)
        score = h.score(distances, cycles)
        if score < best_score:
            cycle1, cycle2 = cycles
            best_score = score
    return cycle1, cycle2    

In [ ]:
points = h.load_file('kroA200.tsp')
distances = h.get_distances(points)

(c1, c2), t = msls(points, distances, 100)
print(t)
print(h.score(distances, (c1, c2)))
h.draw_path(points, c1, c2)


# c1, c2 = ils_small_perturbation(points, distances, t)
# print(h.score(distances, (c1, c2)))
# h.draw_path(points, c1, c2)

# c1, c2 = ils_small_perturbation(points, distances, t)
# print(h.score(distances, (c1, c2)))
# h.draw_path(points, c1, c2)

In [27]:
print(c1, c2)

[32, 99, 73, 56, 35, 174, 9, 13, 191, 92, 105, 148, 189, 109, 28, 183, 60, 135, 31, 23, 158, 173, 120, 171, 45, 11, 146, 39, 131, 110, 116, 114, 52, 0, 84, 144, 190, 26, 192, 157, 76, 160, 5, 108, 106, 156, 46, 30, 176, 12, 78, 66, 119, 111, 154, 74, 53, 124, 180, 1, 34, 172, 167, 184, 61, 82, 128, 102, 113, 97, 87, 147, 55, 151, 177, 70, 37, 38, 129, 71, 49, 94, 93, 90, 149, 163, 139, 20, 153, 88, 40, 58, 2, 72, 188, 141, 130, 179, 155, 80] [98, 107, 68, 166, 29, 67, 168, 22, 143, 101, 69, 75, 181, 194, 112, 175, 132, 136, 42, 104, 4, 195, 85, 138, 27, 199, 170, 140, 57, 33, 89, 142, 24, 16, 145, 7, 133, 21, 182, 126, 185, 134, 41, 54, 19, 63, 161, 159, 14, 122, 197, 64, 186, 150, 79, 127, 59, 100, 3, 162, 48, 17, 36, 137, 25, 198, 6, 81, 77, 8, 123, 117, 15, 178, 65, 152, 43, 187, 62, 50, 193, 115, 121, 169, 51, 10, 83, 47, 165, 86, 125, 95, 164, 103, 96, 44, 196, 118, 91, 18]
